<a href="https://colab.research.google.com/github/Lifeisbetterwithlena/SCM_Fallstudie/blob/main/1_Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Flink
## Standortentscheidung

## Modell

### Indexmengen
$s \in S$ : Menge der potenziellen DarkStores (Standorte)

$i \in I$ : Menge der (Nachfrage) i-Koordinaten 

$j \in J$ : Menge der (Nachfrage) j-Koordinaten  



### Parameter
$n_{ij}$ : Nachfrage an $i$$j$

$u_{s}$ : Lagerumschlagsleistung an Standort $s$

$c_{s}$ : Errichtungskosten für Standort $s$

$ki_{s}$ : $i$-Kooridnate von Standort $s$ 

$kj_{s}$ : $j$-Kooridnate von Standort $s$ 

### Entscheidungsvariablen

$V_{sij} \in \{0,1\}$ : Binäre Versorgungsvariable

$Y_{s} \in \{0,1\}$ : Binäre Standortausbauvariable

### Zielfunktion
Max $NA =   \sum_{s,i,j} (V_{s,i,j} * n_{i,j})$   

### Nebenbedingungen

**(1) Budget einhalten**

$\sum_{s} (c_s*Y_s) \le 1.000.000 $


Kosten  für Ausbau * Entscheidung Ausbau (1/0) darf nicht über 1 mio. liegen

**(2) Lieferzeit einhalten**

$ (|ki_s - i| + |kj_s - j|)* V_{s,i,j} <= 5$

$∀ s,i,j$






**(3) Keine Doppelbedienung der Quandranden**

$\sum_s V_{s,i,j} \le 1$

$∀ i, j$

Prüfe für alle Koordinaten, ob Summe der Standort-Versorgungsvariable kleiner gleich 1 ist

**(4) Kapazitäten einhalten**

$\sum_{i,j}(V_{s,i,j}* n_{i,j}) \le u_s *Y_s $ 

$∀ s$

Prüfe für jeden Standort:

Summe Standort-Versorgungsvariable (1/0) * Nachfrage ist kleiner gleich Umschlagsleistung im jeweiligen Standort

## Implementierung

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Datenaufbereitung


1.   Fallstudien-Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/Fallstudie

In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/Fallstudie && pwd

In [ ]:
# Daten laden
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/Fallstudie"

In [ ]:
# Nachfragedaten lesen & speichern
nachfrage_df = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

In [ ]:
# Nachfragedaten ausgeben
nachfrage_df

In [ ]:
# Transformiere den nachfrage_df in ein Dictionary. Der erste Key sind die Werte von Spalte 1 (i-Koordinate), 
# der zweite Key sind die Werte der Spalten (j-Koordinate)
# Überspringe den ersten Spaltennamen, da hier keine j-Kooridnate enthalten ist

nachfrage = {int(row[0]): {int(col): value for col, value in row.items() if col != 'Unnamed: 0'} for _, row in nachfrage_df.iterrows()}

# Test

# nachfrage i=0 & j=7
print(nachfrage[0][7])

#nachfrage i=10 & j=12
print(nachfrage[10][12])

#nachfrage i=12 & j=12
print(nachfrage[12][12])


In [ ]:
# Standortdaten lesen & speichern
standorte = pd.read_csv(f"{path}/Standorte.csv", sep=";", decimal=",")

In [ ]:
# Standortdaten ausgeben
standorte

## Indexmengen

In [ ]:
# S = Menge der Standorte
S = standorte["Potenzielle_Standorte"].unique().tolist() 


In [ ]:
# Ausgabe Standorte
S

In [ ]:
# I-Kooritnaten
I = [key for key, value in nachfrage.items()]

# Ausgabe I-Koordinaten
print('I-Koordinaten: ' + str(I))

In [ ]:
# J-Koordinaten
J = [list(value.keys()) for value in nachfrage.values()][0]

# Ausgabe I-Kooridnaten
print('J-Koordinaten: ' + str(J))


## Parameter

In [ ]:
# Standortkoordinaten

# I-Koordinate von Standort S
ki = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["i_Koordinate"]

# J-Koordinate von Standort S
kj = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["j_Koordinate"]

# check
print(ki)

print(kj)

In [ ]:
# Umschlagsleistung
u = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["Lagerumschlagleistung"]

#check
u


In [ ]:
# Errichtungskosten
c = standorte.set_index(["Potenzielle_Standorte"]).to_dict("dict")["Errichtungskosten"]

#check
c

In [ ]:
# Nachfrage
n = nachfrage

#check
n[0][0]

## Entscheidungsvariablen

In [ ]:
# Binäre Versorgungsvariable
V={}
for s in S: 
  for i in I:
    for j in J:
        V[s,i,j] = solver.BoolVar(f"{s},{i},{j}")

#check
V[1,3,7]

In [ ]:
# Binäre Standortausbauvariable
Y={}
for s in S:
  Y[s] = solver.BoolVar(f"{s}")


#check
Y[1]

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

## Zielfunktion


# neue Zielfunktion

Maximiere Nachfrageabdeckung:

Max $NA = \sum_{s,i,j} (V_{s,i,j} * n_{i,j})$   



In [ ]:

solver.Maximize(
      sum(V[s,i,j]* n[i][j] for s in S for i in I for j in J)     
  )



## Nebenbedingungen

**(1) Budget einhalten**

$\sum_{s} (c_s*Y_s) \le 1.000.000 $

Summe über: 

Kosten  für Ausbau * Entscheidung Ausbau <= Budget

In [ ]:

solver.Add(sum(c[s]*Y[s] for s in S)<= 1000000)
 

**(2) Lieferzeit einhalten**

$  (|ki_s - i| + |kj_s - j|)* V_{s,i,j}  <= 5$

$∀ s,i,j$






In [ ]:
# Lieferzeit 20 = 13
# Lieferzeit 30 = 21
max_entfernung = 5

In [ ]:
for s in S:
 for i in I:
    for j in J:
      solver.Add((abs(ki[s]-i) +abs(kj[s]- j))*V[s,i,j]<=int(max_entfernung))


**(3) Keine Doppelbedienung der Quandranden**

$\sum_s V_{s,i,j} \le 1$

$∀ i, j$

Prüfe für alle Koordinaten, ob Belieferung kleiner gleich 1.

In [ ]:
for i in I:
  for j in J:
    solver.Add(sum(V[s,i,j] for s in S)<=1)

**(4) Kapazitäten einhalten**

$\sum_{i,j} (V_{s,i,j}* n_{i,j}) \le u_s *Y_s $

$∀ s$

Prüfe für jeden Standort:

Summe Versorgung (ja/nein) * Nachfrage  ist kleiner (gleich) als Umschlagsleistung. 

In [ ]:
for s in S:
  solver.Add(sum(V[s,i,j]*n[i][j] for i in I for j in J)<= (u[s]* Y[s]))


## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
  print('LÖSUNG:')
  print('Zielfunktionswert (Nachfrageabdeckung) =', solver.Objective().Value())
else:
  print('Problem hat keine Lösung')

### Ausgabe Standortausbau

In [ ]:
for s in S:
  if(Y[s].solution_value()==1):
    print("Wir bauen Standort " + str(s) + " aus.")

### Ausgabe Ausbaukosten

In [ ]:
cost = 0

for s in S:
  if(Y[s].solution_value()==1):
    cost = c[s] + cost

print("Unsere Ausbaukosten liegen insgesamt bei " + str(cost) + "€.")

### Belieferungsplan 

In [ ]:
na= 0

for s in S:
  if( Y[s].solution_value()==1):
    print("")
    print("")
    print("Belieferung durch Standort " + str(s) +":")
    print("")
    na_s = 0
    for i in I:
      for j in J:
        if(V[s,i,j].solution_value()==1):
          abstand= (abs(ki[s]-i) +abs(kj[s]- j))
          print("Quadrand: " +str(i) + "/" +str(j) + " Abstand: " + str(abstand) + " Nachfrageabdeckung: " + str(n[i][j]))
          na_s += n[i][j]
    print("")
    print("Kapazität am Standort: " + str(u[s]))
    print("Nachfrageabdeckung durch Standort: " + str(na_s))
    na += na_s

print("")
print("Nachfrageabdeckung insgesamt: " +str(na))

          

### Standortauslastung

In [ ]:
for s in S:
  if(Y[s].solution_value()==1):
    na_s= 0
    kap = 0
    for i in I:
      for j in J:
        if(V[s,i,j].solution_value()==1):
          na_s += n[i][j]
              
    kap= round((na_s*100)/u[s],0)
    print("Der Standort "+str(s) + " ist zu " + str(kap) + "% ausgelastet")